In [1]:
import pandas as pd
import rasa
import os

In [2]:
BASE_PATH = "."

In [3]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:  
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/NLP/CA6"

In [4]:
TRAIN_PATH = os.path.join(BASE_PATH, "Data/MCI_Internet_TrainData.xlsx")
TEST_PATH = os.path.join(BASE_PATH, "Data/MCI_Internet_TestData.xlsx")
RASA_PATH = os.path.join(BASE_PATH, "Rasa")
RASA_DATA_PATH = os.path.join(RASA_PATH, "data")

In [5]:
# Read xlsx file into a pandas dataframe
train_df = pd.read_excel(TRAIN_PATH)
test_df = pd.read_excel(TEST_PATH)


In [6]:
train_df.head()

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,سطح ۴,سطح ۵,پرسش ۱,پرسش ۲,پرسش ۳,...,پرسش ۶,پرسش ۷,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,...,چه کدی برای فعالسازی سرویس اینترنت gprs باید ...,gprs رو با استفاده از کد دستوری چجوری فعال کنم؟,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...
1,2,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,برای فعالسازی سرویس gprs از طریق پیامک چیکار کنم؟,برای فعال سازی سرویس gprs به چه شماره ای باید ...,برای فعالسازی سرویس gprs به چه شماره ای باید ا...,...,به چه شماره ای برای فعالسازی سرویس اینترنت gpr...,gprs رو از طریق پیامک چجوری فعال کنم؟,سرویس اینترنت فعال سازیش از طریق پیامک Sms چجو...,sms فعال سازی سرویس اینترنت چیه؟,به چه شماره ای پیامک اس ام اس sms بدم تا سرویس...,میخوام سرویس اینترنت gprs فعال بشه از طریق پیا...,سرویس اینترنتم از طریق پیامک چیجوری فعال میشه؟,واسه فعال شدن سرویس اینترنتم از طریق پیامک اس ...,برای فعال شدن سرویس اینترنت چه اس ام اسی باید ...,برای فعال شدن اینترنت gprs از طریق پیامک کاری...
2,3,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (G...,اینترنت,مشکلات,غیر فعال سازی سرویس,NaN,NaN,برای غیر فعالسازی سرویس gprs از طریق کد دستوری...,برای غیر فعالسازی سرویس gprs چه شماره ای باید ...,برای غیر فعالسازی سرویس gprs چه کدی رو باید بزنم؟,...,چه کدی برای غیر فعالسازی سرویس اینترنت gprs با...,gprs رو از طریق کد دستوری چجوری غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق کد دستوری ...,کد غیر فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق کد د...,میخوام سرویس اینترنت gprs غیر فعال بشه از طریق...,سرویس اینترنتم چیجوری از طریق کد دستوری غیر فع...,واسه غیر فعال شدن سرویس اینترنتم از طریق کد دس...,برای غیر فعال شدن سرویس اینترنت چه کدی رو باید...,برای غیر فعال شدن اینترنت gprs از طریق کد دستو...
3,4,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (GP...,اینترنت,مشکلات,غیر فعال سازی سرویس,NaN,NaN,برای غیر فعال سازی سرویس gprs از طریق پیامک چ...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,...,به چه شماره ای برای غیر فعالسازی سرویس اینترنت...,gprs رو چجوری از طریق پیامک غیر فعال کنم؟,سرویس اینترنت غیر فعالسازیش از طریق پیامک SMS‌...,sms غیر فعال سازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق پیام...,میخوام سرویس اینترنت gprs غیر فعال بشه باید چه...,سرویس اینترنتم چیجوری از طریق ارسال پیامک SMS‌...,واسه غیر فعال شدن سرویس اینترنتم از طریق پیامک...,برای غیر فعال شدن سرویس اینترنت چه اس ام اسی ب...,برای غیر فعال شدن اینترنت gprs از طریق پیامک ...
4,5,بعد از اقدام و دریافت پیامک تایید فعالسازی سرو...,اینترنت,مشکلات,فعال سازی سرویس,NaN,NaN,بعد از دریافت پیامک تایید فعالسازی اینترنت بای...,بعد از دریافت پیامک تایید فعالسازی اینترنت چیک...,پیامک فعالسازی اینترنت دریافت کردم ولی هنوز ای...,...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت چر...,بعد آمدن پیامک تایید فعالسازی سرویس اینترنت هن...,باید بعد از دریافت پیامک فعالسازی اینترنت کاری...,پیامک تایید فعالسازی سرویس اینترنم اومده ولی ه...,آیا بعد از دریافت پیامک تایید فعالسازی اینترنت...,پیامک تایید فعالسازی اینترنتم اومده و هنوز نت...,پیامکی مبنی بر فعالسازی اینترنت اومد بعدش باید...,پیامکی مبنی بر فعالسازی اینترنت اومده حالا چیک...,اس ام اس تایید فعال سازی اینترنت اومد ولی نت ن...,اس ام اس اومد فعال سازی سرویس اینترنت شما تایی...


In [7]:
deleted_cols = ["سطح ۴","سطح ۵"]
train_df.drop(deleted_cols, axis=1, inplace=True)


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   شناسه    50 non-null     int64 
 1   پاسخ     50 non-null     object
 2   سطح ۱    50 non-null     object
 3   سطح ۲    50 non-null     object
 4   سطح ۳    50 non-null     object
 5   پرسش ۱   50 non-null     object
 6   پرسش ۲   50 non-null     object
 7   پرسش ۳   50 non-null     object
 8   پرسش ۴   50 non-null     object
 9   پرسش ۵   50 non-null     object
 10  پرسش ۶   50 non-null     object
 11  پرسش ۷   50 non-null     object
 12  پرسش ۸   50 non-null     object
 13  پرسش ۹   50 non-null     object
 14  پرسش ۱۰  50 non-null     object
 15  پرسش ۱۱  50 non-null     object
 16  پرسش ۱۲  50 non-null     object
 17  پرسش ۱۳  50 non-null     object
 18  پرسش ۱۴  50 non-null     object
 19  پرسش ۱۵  50 non-null     object
dtypes: int64(1), object(19)
memory usage: 7.9+ KB


In [28]:
intent_cols = ["سطح ۱", "سطح ۲" , "سطح ۳"]
intents = []
intent_questions = {}

# For Each row in train_df, concatenate the columns of intent_cols, and append them intents list. then scan the intents list and if you see any duplicates, prefix them with numbers

for index, row in train_df.iterrows():
    intent = ""
    for col in intent_cols:
        intent += f"{row[col]}_"
    intents.append(intent)

In [29]:
# Break intents list into the sub-lists of duplicate items, for example [x,y,y,x,z] will become [x,x] , [y,y] , [z]
intent_groups = {}
for intent in intents:
    if intent in intent_groups.keys():
        intent_groups[intent].append(f"{intent}_{len(intent_groups[intent])+1}")
    else:
        intent_groups[intent] = [f"{intent}_1"]
            

In [30]:
intent_groups

{'اینترنت_مشکلات_فعال سازی سرویس_': ['اینترنت_مشکلات_فعال سازی سرویس__1',
  'اینترنت_مشکلات_فعال سازی سرویس__2',
  'اینترنت_مشکلات_فعال سازی سرویس__3'],
 'اینترنت_مشکلات_غیر فعال سازی سرویس_': ['اینترنت_مشکلات_غیر فعال سازی سرویس__1',
  'اینترنت_مشکلات_غیر فعال سازی سرویس__2'],
 'اینترنت_مشکلات_سرویس آموزش_': ['اینترنت_مشکلات_سرویس آموزش__1',
  'اینترنت_مشکلات_سرویس آموزش__2'],
 'اینترنت_مشکلات_مشکلات اتصال به اینترنت_': ['اینترنت_مشکلات_مشکلات اتصال به اینترنت__1',
  'اینترنت_مشکلات_مشکلات اتصال به اینترنت__2',
  'اینترنت_مشکلات_مشکلات اتصال به اینترنت__3',
  'اینترنت_مشکلات_مشکلات اتصال به اینترنت__4',
  'اینترنت_مشکلات_مشکلات اتصال به اینترنت__5',
  'اینترنت_مشکلات_مشکلات اتصال به اینترنت__6'],
 'اینترنت_مشکلات_4G و 4.5G_': ['اینترنت_مشکلات_4G و 4.5G__1',
  'اینترنت_مشکلات_4G و 4.5G__2',
  'اینترنت_مشکلات_4G و 4.5G__3',
  'اینترنت_مشکلات_4G و 4.5G__4'],
 'اینترنت_بسته_بسته\u200cهای اینترنت_': ['اینترنت_بسته_بسته\u200cهای اینترنت__1',
  'اینترنت_بسته_بسته\u200cهای اینترنت__2',
  'این

In [25]:
# write intent_groups into intent_group.txt
with open(os.path.join(BASE_PATH, "intent_groups.txt"), "w") as f:
    for intent_group in intent_groups.values():
        for intent in intent_group:
            f.write(f"{intent}\t")
        f.write("\n")

In [33]:
intent_group_count = {}
for intent_group, list in intent_groups.items():
    intent_group_count[intent_group] = len(list)
    

In [34]:
intent_group_count

{'اینترنت_مشکلات_فعال سازی سرویس_': 3,
 'اینترنت_مشکلات_غیر فعال سازی سرویس_': 2,
 'اینترنت_مشکلات_سرویس آموزش_': 2,
 'اینترنت_مشکلات_مشکلات اتصال به اینترنت_': 6,
 'اینترنت_مشکلات_4G و 4.5G_': 4,
 'اینترنت_بسته_بسته\u200cهای اینترنت_': 3,
 'اینترنت_بسته_خرید بسته از شاپ (فروشگاه)_': 2,
 'اینترنت_بسته_خرید بسته از *100#_': 1,
 'اینترنت_بسته_نام تجاری نوترینو_': 2,
 'اینترنت_بسته_بسته\u200cهای اینترنت پاک_': 1,
 'اینترنت_بسته_بسته\u200cهای ساعات مشخص_': 5,
 'اینترنت_بسته_بسته ویژه مشترکین جدید_': 3,
 'اینترنت_بسته_استعلام بسته_': 4,
 'اینترنت_بسته_لغو تمدید بسته_': 7,
 'اینترنت_بسته_بسته\u200cهای نامحدود شبانه_': 5}

In [37]:
# for each row of train_df, make a column called intent_main which is intent += f"{row[col]}_" for all col in intent_cols
train_df["intent_main"] = ""
for index, row in train_df.iterrows():
    intent = ""
    for col in intent_cols:
        intent += f"{row[col]}_"
    train_df.loc[index, "intent_main"] = intent


In [39]:
train_df.head(1)

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,پرسش ۱,پرسش ۲,پرسش ۳,پرسش ۴,پرسش ۵,...,پرسش ۷,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵,intent_main
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs با استفاده از کد دستو...,چه شماره ای برای فعالسازی سرویس اینترنت gprs ب...,...,gprs رو با استفاده از کد دستوری چجوری فعال کنم؟,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...,اینترنت_مشکلات_فعال سازی سرویس_


In [ ]:
# Count the accumulated number of items in all values of intent_groups dictionary
intent_count = 0
for key in intent_groups.keys():
    intent_count += len(intent_groups[key])

intent_count

In [40]:
train_df["intent_postfixed"] = ""

In [41]:
intent_count_tmp = {}
for index,row in train_df.iterrows():
    intent = row["intent_main"]
    intent_postfixed = ""
    if intent in intent_groups.keys():
        if intent not in intent_count_tmp.keys():
            intent_count_tmp[intent] = 0
            intent_postfixed = intent_groups[intent][0]
        else:
            intent_count_tmp[intent] += 1
            intent_postfixed = intent_groups[intent][intent_count_tmp[intent]]
    train_df.loc[index, "intent_postfixed"] = intent_postfixed

In [43]:
train_df.head(10)

,شناسه,پاسخ,سطح ۱,سطح ۲,سطح ۳,پرسش ۱,پرسش ۲,پرسش ۳,پرسش ۴,پرسش ۵,...,پرسش ۸,پرسش ۹,پرسش ۱۰,پرسش ۱۱,پرسش ۱۲,پرسش ۱۳,پرسش ۱۴,پرسش ۱۵,intent_main,intent_postfixed
0,1,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,برای فعال سازی سرویس gprs با استفاده از کد دس...,برای فعالسازی سرویس gprs چه شماره ای باید بگیرم؟,برای فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs با استفاده از کد دستو...,چه شماره ای برای فعالسازی سرویس اینترنت gprs ب...,...,سرویس اینترنت فعال سازیش با استفاده از کد دستو...,کد فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم با استفاده ا...,میخوام سرویس اینترنت gprs فعال بشه با استفاده ...,سرویس اینترنتم چی جوری با استفاده از کد دستوری...,واسه فعال شدن سرویس اینترنتم با کد دستوری چیکا...,برای فعال شدن سرویس اینترنت چه کدی رو باید بگیرم؟,برای فعال شدن اینترنت gprs با استفاده از کد دس...,اینترنت_مشکلات_فعال سازی سرویس_,اینترنت_مشکلات_فعال سازی سرویس__1
1,2,مشترک گرامی جهت فعالسازی سرویس اینترنت (GPRS) ...,اینترنت,مشکلات,فعال سازی سرویس,برای فعالسازی سرویس gprs از طریق پیامک چیکار کنم؟,برای فعال سازی سرویس gprs به چه شماره ای باید ...,برای فعالسازی سرویس gprs به چه شماره ای باید ا...,چجوری سرویس اینترنت gprs از طریق پیامک اس ام ا...,به کجا برای فعال سازی سرویس اینترنت gprs باید ...,...,سرویس اینترنت فعال سازیش از طریق پیامک Sms چجو...,sms فعال سازی سرویس اینترنت چیه؟,به چه شماره ای پیامک اس ام اس sms بدم تا سرویس...,میخوام سرویس اینترنت gprs فعال بشه از طریق پیا...,سرویس اینترنتم از طریق پیامک چیجوری فعال میشه؟,واسه فعال شدن سرویس اینترنتم از طریق پیامک اس ...,برای فعال شدن سرویس اینترنت چه اس ام اسی باید ...,برای فعال شدن اینترنت gprs از طریق پیامک کاری...,اینترنت_مشکلات_فعال سازی سرویس_,اینترنت_مشکلات_فعال سازی سرویس__2
2,3,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (G...,اینترنت,مشکلات,غیر فعال سازی سرویس,برای غیر فعالسازی سرویس gprs از طریق کد دستوری...,برای غیر فعالسازی سرویس gprs چه شماره ای باید ...,برای غیر فعالسازی سرویس gprs چه کدی رو باید بزنم؟,چجوری سرویس اینترنت gprs رو از طریق کد دستوری ...,چه شماره ای برای غیر فعالسازی سرویس اینترنت gp...,...,سرویس اینترنت غیر فعالسازیش از طریق کد دستوری ...,کد غیر فعالسازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق کد د...,میخوام سرویس اینترنت gprs غیر فعال بشه از طریق...,سرویس اینترنتم چیجوری از طریق کد دستوری غیر فع...,واسه غیر فعال شدن سرویس اینترنتم از طریق کد دس...,برای غیر فعال شدن سرویس اینترنت چه کدی رو باید...,برای غیر فعال شدن اینترنت gprs از طریق کد دستو...,اینترنت_مشکلات_غیر فعال سازی سرویس_,اینترنت_مشکلات_غیر فعال سازی سرویس__1
3,4,مشترک گرامی جهت غیر فعالسازی سرویس اینترنت (GP...,اینترنت,مشکلات,غیر فعال سازی سرویس,برای غیر فعال سازی سرویس gprs از طریق پیامک چ...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,برای غیر فعالسازی سرویس gprs به چه شماره ای با...,چجوری سرویس اینترنت gprs از طریق پیامک SMS‌ رو...,به کجا برای غیر فعالسازی سرویس اینترنت gprs با...,...,سرویس اینترنت غیر فعالسازیش از طریق پیامک SMS‌...,sms غیر فعال سازی سرویس اینترنت چیه؟,چیکار باید بکنم تا سرویس اینترنتم از طریق پیام...,میخوام سرویس اینترنت gprs غیر فعال بشه باید چه...,سرویس اینترنتم چیجوری از طریق ارسال پیامک SMS‌...,واسه غیر فعال شدن سرویس اینترنتم از طریق پیامک...,برای غیر فعال شدن سرویس اینترنت چه اس ام اسی ب...,برای غیر فعال شدن اینترنت gprs از طریق پیامک ...,اینترنت_مشکلات_غیر فعال سازی سرویس_,اینترنت_مشکلات_غیر فعال سازی سرویس__2
4,5,بعد از اقدام و دریافت پیامک تایید فعالسازی سرو...,اینترنت,مشکلات,فعال سازی سرویس,بعد از دریافت پیامک تایید فعالسازی اینترنت بای...,بعد از دریافت پیامک تایید فعالسازی اینترنت چیک...,پیامک فعالسازی اینترنت دریافت کردم ولی هنوز ای...,پیامک فعال سازی اومده ولی هنوز نتم وصل نشده، م...,بعد از آمدن پیامک تایید فعالسازی سرویس اینترنت...,...,باید بعد از دریافت پیامک فعالسازی اینترنت کاری...,پیامک تایید فعالسازی سرویس اینترنم اومده ولی ه...,آیا بعد از دریافت پیامک تایید فعالسازی اینترنت...,پیامک تایید فعالسازی اینترنتم اومده و هنوز نت...,پیامکی مبنی بر فعالسازی اینترنت اومد بعدش باید...,پیامکی مبنی بر فعالسازی اینترنت اومده حالا چیک...,اس ام اس تایید فعال سازی اینترنت او

In [12]:
# Write Every item in intens into a txt file 
for intent in intents:
    with open(os.path.join(BASE_PATH, "intents.txt"), "a") as f:
        f.write(intent + "\n")